# **Proceso ETL de dato COVID-19**
---
Dentro de la empresa es necesario la obtencion de los datos del informe COVID-19 entregado por la autoridad sanitaria, para su posterior analisis a traves de graficos estadisticos.

Los datos obtenidos provienen de repositorios pertenecientes al Ministerio de Ciencia y a Ivan Gonzales, ambos repositorios alojados en GitHub.

In [1]:
import pandas as pd
import datetime as dt
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

### Para comenzar con la obtencion de los datos, es necesario establecer el numero de dias de estudio

In [2]:
periodo = dt.timedelta(days=int(input("Ingrese el numero de dias: ")))

In [3]:
fecha_final = dt.date.today() - dt.timedelta(days=1)
fecha_inicial = fecha_final - periodo
nombre_hoja = "Hoja1"

### Funciones necesarias

*Funcion para transformar fechas en formato string yyyy-mm-dd ; dd-mm-yyyy a formato Date*

In [4]:
def string_date(fecha_str):
  fecha = fecha_str.split("-")
  if (len(fecha[0]) > 2):
    fecha_str = fecha_str.split("-")
    return dt.date(int(fecha_str[0]),int(fecha_str[1]),int(fecha_str[2]))
  return dt.date(int(fecha[2]),int(fecha[1]),int(fecha[0]))


*Funcion para setear los sw a None si una tupla no corresponde al periodo de fechas indicados*

In [5]:
def set_sw(tupla,fecha1,fecha2):
  if string_date(tupla["Fecha"]) < fecha1 or string_date(tupla["Fecha"]) > fecha2:
    tupla["sw"] = None
  return tupla


*Funcion que setea sw en nulo si el nombre de la comuna de la tupla es igual a la palabra ingresada*

In [6]:
def set_comuna(tupla,palabra):
  if tupla["Comuna"].split(" ")[0] == palabra:
    tupla["sw"] = None
  return tupla

## Datos historicos de los casos COVID-19 por comuna.
---



### Luego es necesario ingresar la URL directa del archivo CSV.
Para este caso se ingresa la URL proveniente de un repositorio github del MinCiencia, el archivo se llama "*historial_comunas.csv*"

In [7]:
df_historial_comunas = pd.read_csv('https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/historial_comunas.csv')

*Nota: Si se desea ingresar un archio .CSV local, es necesario alojarlo en el mismo directorio del programa*

*Agrega una columna extra que sirve como switch*

In [8]:
df_historial_comunas = df_historial_comunas.assign(sw=0)

*Se aplica la funcion set_sw en el DF*

In [9]:
df_historial_comunas = df_historial_comunas.apply(set_sw,args=[fecha_inicial,fecha_final],axis=1)

*Se eliminan las tuplas nulas y la columna sw*

In [10]:
df_historial_comunas = df_historial_comunas.dropna()
df_historial_comunas = df_historial_comunas.drop(columns=["sw"])

In [11]:
df_historial_comunas["Fecha"] = df_historial_comunas["Fecha"].apply(string_date)

*Se exportan los datos en un archivo excel*

In [12]:
df_historial_comunas.to_excel("ContagiosDiarios.xlsx",sheet_name=nombre_hoja,index=False)

## Casos activos por comuna
---



### Luego es necesario ingresar la URL directa del archivo CSV.
Para este caso se ingresa la URL proveniente de un repositorio github del MinCiencia, el archivo se llama "*historial_comunas.csv*"

In [13]:
df_casos_activos_comuna = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna_std.csv')

*Nota: Si se desea ingresar un archio .CSV local, es necesario alojarlo en el mismo directorio del programa*

*Agrega una columna extra que sirve como switch*

In [14]:
df_casos_activos_comuna = df_casos_activos_comuna.assign(sw=0)

*Se aplica la funcion set_sw en el DF*

In [15]:
df_casos_activos_comuna = df_casos_activos_comuna.apply(set_sw,args=[fecha_inicial,fecha_final],axis=1)

*Se aplica la funcion set_comuna para eliminar las tuplas con nombre "Desconocido" y "Total"*

In [16]:
df_casos_activos_comuna = df_casos_activos_comuna.apply(set_comuna,args=["Desconocido"],axis=1)
df_casos_activos_comuna = df_casos_activos_comuna.apply(set_comuna,args=["Total"],axis=1)

*Se elimina los datos nulos y la columna sw*

In [17]:
df_casos_activos_comuna = df_casos_activos_comuna.dropna(subset=['sw'])
df_casos_activos_comuna = df_casos_activos_comuna.drop(columns='sw')

*Se ordenan las columnas a un formato predefinido*

In [18]:
df_casos_activos_comuna = df_casos_activos_comuna[["Region","Codigo region","Comuna","Codigo comuna","Poblacion","Casos activos","Fecha"]]

In [19]:
df_casos_activos_comuna.columns = ["Region","Codigo region","Comuna","Codigo comuna","Poblacion","Importe","Mes"]

*Se aplica la funcion string_date() a la columna de fechas*

In [20]:
df_casos_activos_comuna["Mes"] = df_casos_activos_comuna["Mes"].apply(string_date)

*Se exportan los datos en un excel*

In [21]:
df_casos_activos_comuna.to_excel("ActivosDiarios.xlsx",sheet_name=nombre_hoja,index=False)